In [1]:
import numpy as np
import pandas as pd
import json
pd.set_option('display.max_columns', None)
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
#pd.set_option('max_colwidth', None)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,GridSearchCV,RepeatedKFold
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,roc_auc_score,precision_score,precision_recall_curve, auc,roc_curve
from sklearn.ensemble import RandomForestClassifier

## Importing, preliminary cleaning and validation


In [2]:
data = pd.read_json('matches.jsonl',lines=True)


In [3]:
#extracting playerbased data
df = pd.json_normalize(data['info'])['participants'].explode()
df_player = pd.json_normalize(df)
df_player.head(10)

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsOnRecentlyHealedByAramPack,challenges.killsUnderOwnTurret,challenges.killsWithHelpFromEpicMonster,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.lostAnInhibitor,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,ch

In [4]:
#retrieving matchId for some Filtering Purposes
match_id= np.repeat(pd.json_normalize(data['metadata'])['matchId'].values,10,axis=0)

In [5]:
pd.DataFrame(np.repeat(pd.json_normalize(pd.json_normalize(data['info'])['teams'].explode()).values,5,axis=0),
             columns=pd.json_normalize(pd.json_normalize(data['info'])['teams'].explode()).columns)

,bans,teamId,win,objectives.baron.first,objectives.baron.kills,objectives.champion.first,objectives.champion.kills,objectives.dragon.first,objectives.dragon.kills,objectives.inhibitor.first,objectives.inhibitor.kills,objectives.riftHerald.first,objectives.riftHerald.kills,objectives.tower.first,objectives.tower.kills
0,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
1,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
2,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
3,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
4,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83505,"[{'championId': 203, 'pickTurn': 6}, {'champio...",200,True,True,1,True,50,True,3,True,2,False,1,False,9
83506,"[{'championId': 203, 'pickTurn': 6}, {'champio...",200,True,True,1,True,50,True,3,True,2,False,1,False,9
83507,"[{'championId': 203, 'pickTurn': 6}, {'champio...",200,True,True,1,True,50,True,3,True,2,False,1,False,9
83508,"[{'championId': 203, 'pickTurn': 6}, {'champio...",200,True,True,1,True,50,True,3,True,2,False,1,False,9


In [6]:
#extracting teambased data (dragon,braon,firstblood etc..)
#copying the team data x5 so it matches to the player dataset 
df_team= pd.DataFrame(np.repeat(pd.json_normalize(pd.json_normalize(data['info'])['teams'].explode()).values,5,axis=0),
             columns=pd.json_normalize(pd.json_normalize(data['info'])['teams'].explode()).columns)
df_team.head(5)

,bans,teamId,win,objectives.baron.first,objectives.baron.kills,objectives.champion.first,objectives.champion.kills,objectives.dragon.first,objectives.dragon.kills,objectives.inhibitor.first,objectives.inhibitor.kills,objectives.riftHerald.first,objectives.riftHerald.kills,objectives.tower.first,objectives.tower.kills
0,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
1,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
2,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
3,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11
4,"[{'championId': 26, 'pickTurn': 1}, {'champion...",100,True,True,2,False,37,False,2,True,4,True,2,False,11


In [7]:
#cleaning teamdata of unneeded variables(bans are irrelevant for this scope)
df_team = df_team.drop(columns=['bans'])
df_team = df_team.astype('int64')
df_team.head(5)

,teamId,win,objectives.baron.first,objectives.baron.kills,objectives.champion.first,objectives.champion.kills,objectives.dragon.first,objectives.dragon.kills,objectives.inhibitor.first,objectives.inhibitor.kills,objectives.riftHerald.first,objectives.riftHerald.kills,objectives.tower.first,objectives.tower.kills
0,100,1,1,2,0,37,0,2,1,4,1,2,0,11
1,100,1,1,2,0,37,0,2,1,4,1,2,0,11
2,100,1,1,2,0,37,0,2,1,4,1,2,0,11
3,100,1,1,2,0,37,0,2,1,4,1,2,0,11
4,100,1,1,2,0,37,0,2,1,4,1,2,0,11


In [8]:
#Joining to create player dataframe with respective game/team objectives
df_total = pd.concat([df_player,df_team],axis=1)


In [9]:
#adding match_id
df_total['matchId'] = match_id
df_total.head(5)

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsOnRecentlyHealedByAramPack,challenges.killsUnderOwnTurret,challenges.killsWithHelpFromEpicMonster,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.lostAnInhibitor,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,ch

## Data Cleaning

### Games Remade
in league of legends, if a player in either teams is not present for 90 seconds at the start of the game, the team with this player present in will have the option to remake, i would want these games out of the data as the team that initiated the remake counts as loss when in truth its not exactly a loss and that may confuse the model 

In [10]:
#players that were in games that remade 
df_total[df_total['timePlayed'] <195]

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsOnRecentlyHealedByAramPack,challenges.killsUnderOwnTurret,challenges.killsWithHelpFromEpicMonster,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.lostAnInhibitor,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,ch

In [11]:
#removing remade games
df_total = df_total.drop(df_total[df_total['timePlayed'] <195].index,axis=0)

In [12]:
df_total[df_total['timePlayed'] <195]

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsOnRecentlyHealedByAramPack,challenges.killsUnderOwnTurret,challenges.killsWithHelpFromEpicMonster,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.lostAnInhibitor,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,ch

### Games Ended in less than 15 minutes


since there are some challenges that are tracked all the way up to 14 minutes, their importance may be overestimated  for games that lasted less than that 

In [13]:
df_total[df_total['timePlayed']<900]

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsOnRecentlyHealedByAramPack,challenges.killsUnderOwnTurret,challenges.killsWithHelpFromEpicMonster,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.lostAnInhibitor,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,ch

In [14]:
#removing >15 games
df_total = df_total.drop(df_total[df_total['timePlayed'] <900].index,axis=0)

### Missing teamPosition
some values exist with missing team position, ive decided to remove the games affected(10x the rows that appear with no team position), the other reason i elected to remove these games is that it seems these games have afk players, i would rather keep all the data of games with similiar state

In [15]:
#filtering for games where players were missing teamPosition
MissingPos_mId = df_total[df_total['teamPosition']== '']['matchId']
df_total[df_total['teamPosition']== '']

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,championName,championTransform,commandPings,consumablesPurchased,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,dangerPings,deaths,detectorWardsPlaced,doubleKills,dragonKills,eligibleForProgression,enemyMissingPings,enemyVisionPings,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,gameEndedInEarlySurrender,gameEndedInSurrender,getBackPings,goldEarned,goldSpent,holdPings,individualPosition,inhibitorKills,inhibitorTakedowns,inhibitorsLost,item0,item1,item2,item3,item4,item5,item6,itemsPurchased,killingSprees,kills,lane,largestCriticalStrike,largestKillingSpree,largestMultiKill,longestTimeSpentLiving,magicDamageDealt,magicDamageDealtToChampions,magicDamageTaken,needVisionPings,neutralMinionsKilled,nexusKills,nexusLost,nexusTakedowns,objectivesStolen,objectivesStolenAssists,onMyWayPings,participantId,pentaKills,physicalDamageDealt,physicalDamageDealtToChampions,physicalDamageTaken,playerAugment1,playerAugment2,playerAugment3,playerAugment4,playerSubteamId,profileIcon,pushPings,puuid,quadraKills,riotIdName,riotIdTagline,role,sightWardsBoughtInGame,spell1Casts,spell2Casts,spell3Casts,spell4Casts,subteamPlacement,summoner1Casts,summoner1Id,summoner2Casts,summoner2Id,summonerId,summonerLevel,summonerName,teamEarlySurrendered,teamId,teamPosition,timeCCingOthers,timePlayed,totalAllyJungleMinionsKilled,totalDamageDealt,totalDamageDealtToChampions,totalDamageShieldedOnTeammates,totalDamageTaken,totalEnemyJungleMinionsKilled,totalHeal,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,totalUnitsHealed,tripleKills,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,challenges.12AssistStreakCount,challenges.abilityUses,challenges.acesBefore15Minutes,challenges.alliedJungleMonsterKills,challenges.baronBuffGoldAdvantageOverThreshold,challenges.baronTakedowns,challenges.blastConeOppositeOpponentCount,challenges.bountyGold,challenges.buffsStolen,challenges.completeSupportQuestInTime,challenges.controlWardTimeCoverageInRiverOrEnemyHalf,challenges.controlWardsPlaced,challenges.damagePerMinute,challenges.damageTakenOnTeamPercentage,challenges.dancedWithRiftHerald,challenges.deathsByEnemyChamps,challenges.dodgeSkillShotsSmallWindow,challenges.doubleAces,challenges.dragonTakedowns,challenges.earliestBaron,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.effectiveHealAndShielding,challenges.elderDragonKillsWithOpposingSoul,challenges.elderDragonMultikills,challenges.enemyChampionImmobilizations,challenges.enemyJungleMonsterKills,challenges.epicMonsterKillsNearEnemyJungler,challenges.epicMonsterKillsWithin30SecondsOfSpawn,challenges.epicMonsterSteals,challenges.epicMonsterStolenWithoutSmite,challenges.firstTurretKilled,challenges.flawlessAces,challenges.fullTeamTakedown,challenges.gameLength,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.goldPerMinute,challenges.hadOpenNexus,challenges.immobilizeAndKillWithAlly,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.junglerTakedownsNearDamagedEpicMonster,challenges.kTurretsDestroyedBeforePlatesFall,challenges.kda,challenges.killAfterHiddenWithAlly,challenges.killParticipation,challenges.killedChampTookFullTeamDamageSurvived,challenges.killingSprees,challenges.killsNearEnemyTurret,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.killsOnRecentlyHealedByAramPack,challenges.killsUnderOwnTurret,challenges.killsWithHelpFromEpicMonster,challenges.knockEnemyIntoTeamAndKill,challenges.landSkillShotsEarlyGame,challenges.laneMinionsFirst10Minutes,challenges.laningPhaseGoldExpAdvantage,challenges.legendaryCount,challenges.lostAnInhibitor,challenges.maxCsAdvantageOnLaneOpponent,challenges.maxKillDeficit,ch

In [17]:
#dropping the games with affected players
df_total = df_total.drop(df_total[df_total['matchId'].isin(np.array(MissingPos_mId))].index,axis=0)

In [20]:
#saving cleaned data for EDA
df_total.to_csv('clean_data.csv',index=False)

## Column Filtering/Data Engineering

seeing as most of the features are captured for the duration of the whole game, i tried to pick exclusively the features that log the data up to 10/15 minutes i order to emulate the situation where i am predicting a game's outcome as it is happening.
all of the data about the selected columns can be found at

https://challenges.darkintaqt.com/challenges and 

https://leagueoflegends.fandom.com/wiki/Challenges_(League_of_Legends)#List

In [17]:
'challenges.acesBefore15Minutes' #a feature that logs the amount of aces(total opposite team elemination) before the minute 15
#counts(discrete)for all team(mode)

'challenges.baronBuffGoldAdvantageOverThreshold' #from the wiki page "Generate a 2500 gold advantage during your Baron Buff"
#counts(discrete) for all team(mode)

'challenges.fasterSupportQuestCompletion' #tells which support completed their quest faster than their lane counterpart
#counts only for players(boolean)(sum)

'challenges.completeSupportQuestInTime' #support quests are usually done at best by somewhere in the range of 5 to 7 minutes at worst they can go to something like 15 minutes hardly ever more than that
#counts only for players(boolean)(sum)
    
'challenges.earlyLaningPhaseGoldExpAdvantage' #from the wiki page 'End the early laning phase (7 minutes) with 20% more gold and experience than your role opponent'
#counts only for players(boolean)(sum)         

'challenges.laningPhaseGoldExpAdvantage' # same as above but 14 minutes instead of 7
#counts only for players(boolean)(sum)   

'challenges.getTakedownsInAllLanesEarlyJungleAsLaner' #from the wiki page "Get takedowns on enemy champions before jungle camps spawn (1:30)"
#counts for players but concerns the whole team(discrete)(mode)       #given the nature of master ranked games, this might be too sparse in the data


'challenges.killsOnOtherLanesEarlyJungleAsLaner'#same as above but continous instead of boolean, takedowns measure both kills and assists in boolean
#counts for players(sum)                        #this measures the kills in their quantity
    


'challenges.initialBuffCount' #i believe that this feature explain the challenge 'Three Buffed' and its about getting the majority of early camps (3 out of 4)
#counts for players(sum)                             #given the natrue of master ranked games, this might be too sparse in the data
    
    
'challenges.initialCrabCount' # i believe this feature explains the challenge 'crab wrangler' and its about getting the majority of early scuttle crabs (2 out of 2)
#counts for players(sum)


'challenges.jungleCsBefore10Minutes' #this feature is challenge for getting 70 creep score before minute 10
#counts for players(sum)              #from the wiki page 'As jungler, get 70 CS from jungle monsters before 10 minutes'

'challenges.laneMinionsFirst10Minutes'# same as above but for laners instead of junglers    
#counts for players(sum)   
    
    
'challenges.kTurretsDestroyedBeforePlatesFall'#this feature is a challenge for killing turrets before plates fall(at somewhere around 13 minutes)
#counts for players(sum)


'challenges.turretPlatesTaken'# this feature logs the total amount of turret plates taken
#counts for players(sum)


'challenges.firstTurretKilledTime' #this feature logs the time at which the first turret fell
#counts for team(mode)


'challenges.takedownsFirstXMinutes' # cant be completely sure about the meaning of this feature
 #counts for players(sum)                                   #from the wiki page "Get 12 Takedowns (kill or assist) on enemy champions by 15 minutes"
    
'challenges.junglerKillsEarlyJungle' #from the wikipage "As jungler, get kills on the enemy jungler in their own jungle before 10 minutes"

'challenges.killsOnLanersEarlyJungleAsJungler' # this feature is self explanatory

'challenges.killsOnLanersEarlyJungleAsJungler'

In [18]:
colms= ['teamId','matchId','challenges.acesBefore15Minutes','challenges.baronBuffGoldAdvantageOverThreshold','challenges.fasterSupportQuestCompletion',
'challenges.completeSupportQuestInTime','challenges.earlyLaningPhaseGoldExpAdvantage','challenges.laningPhaseGoldExpAdvantage',
'challenges.getTakedownsInAllLanesEarlyJungleAsLaner','challenges.killsOnOtherLanesEarlyJungleAsLaner',
'challenges.initialBuffCount','challenges.initialCrabCount','challenges.jungleCsBefore10Minutes','challenges.laneMinionsFirst10Minutes',
'challenges.kTurretsDestroyedBeforePlatesFall','challenges.turretPlatesTaken','challenges.firstTurretKilledTime',
'challenges.takedownsFirstXMinutes','challenges.junglerKillsEarlyJungle','challenges.killsOnLanersEarlyJungleAsJungler',
]


df_total = df_total[colms]

#for some reason teamid shows up duplicated
df_total = df_total.iloc[:,1:]

#changing teamid datatype
df_total['teamId'] = df_total['teamId'].apply(lambda x: 'Blue' if x == 100 else 'Red')

In [19]:
#filling nans with 0, nans here are equivlant for 0
df_total = df_total.fillna(0)

In [20]:
df_total

,teamId,matchId,challenges.acesBefore15Minutes,challenges.baronBuffGoldAdvantageOverThreshold,challenges.fasterSupportQuestCompletion,challenges.completeSupportQuestInTime,challenges.earlyLaningPhaseGoldExpAdvantage,challenges.laningPhaseGoldExpAdvantage,challenges.getTakedownsInAllLanesEarlyJungleAsLaner,challenges.killsOnOtherLanesEarlyJungleAsLaner,challenges.initialBuffCount,challenges.initialCrabCount,challenges.jungleCsBefore10Minutes,challenges.laneMinionsFirst10Minutes,challenges.kTurretsDestroyedBeforePlatesFall,challenges.turretPlatesTaken,challenges.firstTurretKilledTime,challenges.takedownsFirstXMinutes,challenges.junglerKillsEarlyJungle,challenges.killsOnLanersEarlyJungleAsJungler
0,Blue,NA1_4698306968,0,2.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,63,0,0,0.0,0,0.0,0.0
1,Blue,NA1_4698306968,0,2.0,0.0,0,0.0,0.0,0.0,0.0,2,1,63.2,2,0,3,0.0,9,0.0,2.0
2,Blue,NA1_4698306968,0,2.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,69,0,1,0.0,5,0.0,0.0
3,Blue,NA1_4698306968,0,2.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,69,1,2,0.0,6,0.0,0.0
4,Blue,NA1_4698306968,0,2.0,1.0,1,0.0,0.0,0.0,0.0,0,0,0.0,16,0,3,0.0,7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83505,Red,NA1_4699607377,0,1.0,0.0,0,0.0,1.0,0.0,1.0,0,0,0.0,54,0,2,0.0,7,0.0,0.0
83506,Red,NA1_4699607377,0,1.0,0.0,0,0.0,0.0,0.0,0.0,2,1,63.0,0,0,0,0.0,5,0.0,0.0
83507,Red,NA1_4699607377,0,1.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,71,0,1,0.0,1,0.0,0.0
83508,Red,NA1_4699607377,0,1.0,0.0,0,1.0,1.0,0.0,1.0,0,0,0.0,78,1,5,0.0,13,0.0,0.0


In [21]:
val_agg= ['challenges.acesBefore15Minutes','challenges.baronBuffGoldAdvantageOverThreshold','challenges.fasterSupportQuestCompletion',
'challenges.completeSupportQuestInTime','challenges.earlyLaningPhaseGoldExpAdvantage','challenges.laningPhaseGoldExpAdvantage',
'challenges.getTakedownsInAllLanesEarlyJungleAsLaner','challenges.killsOnOtherLanesEarlyJungleAsLaner',
'challenges.initialBuffCount','challenges.initialCrabCount','challenges.jungleCsBefore10Minutes','challenges.laneMinionsFirst10Minutes',
'challenges.kTurretsDestroyedBeforePlatesFall','challenges.turretPlatesTaken','challenges.firstTurretKilledTime',
'challenges.takedownsFirstXMinutes','challenges.junglerKillsEarlyJungle','challenges.killsOnLanersEarlyJungleAsJungler',
]

In [22]:
df_total_pv= df_total.pivot_table(index='matchId',columns='teamId',values=val_agg,aggfunc={'challenges.acesBefore15Minutes':'first',
                                                               'challenges.baronBuffGoldAdvantageOverThreshold':'first',
                                                               'challenges.fasterSupportQuestCompletion':'sum',
                                                               'challenges.completeSupportQuestInTime':'sum',
                                                               'challenges.earlyLaningPhaseGoldExpAdvantage':'sum',
                                                               'challenges.laningPhaseGoldExpAdvantage':'sum',
                                                               'challenges.getTakedownsInAllLanesEarlyJungleAsLaner':'max',
                                                               'challenges.killsOnOtherLanesEarlyJungleAsLaner':'sum',
                                                               'challenges.initialBuffCount':'sum',
                                                               'challenges.initialCrabCount':'sum',
                                                               'challenges.jungleCsBefore10Minutes':'sum',
                                                               'challenges.laneMinionsFirst10Minutes':'sum',
                                                               'challenges.kTurretsDestroyedBeforePlatesFall':'sum',
                                                               'challenges.turretPlatesTaken':'sum',
                                                               'challenges.firstTurretKilledTime':'first',
                                                               'challenges.takedownsFirstXMinutes':'sum',
                                                               'challenges.junglerKillsEarlyJungle':'sum',
                                                               'challenges.killsOnLanersEarlyJungleAsJungler':'sum'})

In [23]:
#flattening the multi-index columns
df_total_pv.columns = ['_'.join(df_total_pv.columns[col]) for col in np.arange(len(df_total_pv.columns))]


In [24]:
df_total_pv

,challenges.acesBefore15Minutes_Blue,challenges.acesBefore15Minutes_Red,challenges.baronBuffGoldAdvantageOverThreshold_Blue,challenges.baronBuffGoldAdvantageOverThreshold_Red,challenges.completeSupportQuestInTime_Blue,challenges.completeSupportQuestInTime_Red,challenges.earlyLaningPhaseGoldExpAdvantage_Blue,challenges.earlyLaningPhaseGoldExpAdvantage_Red,challenges.fasterSupportQuestCompletion_Blue,challenges.fasterSupportQuestCompletion_Red,challenges.firstTurretKilledTime_Blue,challenges.firstTurretKilledTime_Red,challenges.getTakedownsInAllLanesEarlyJungleAsLaner_Blue,challenges.getTakedownsInAllLanesEarlyJungleAsLaner_Red,challenges.initialBuffCount_Blue,challenges.initialBuffCount_Red,challenges.initialCrabCount_Blue,challenges.initialCrabCount_Red,challenges.jungleCsBefore10Minutes_Blue,challenges.jungleCsBefore10Minutes_Red,challenges.junglerKillsEarlyJungle_Blue,challenges.junglerKillsEarlyJungle_Red,challenges.kTurretsDestroyedBeforePlatesFall_Blue,challenges.kTurretsDestroyedBeforePlatesFall_Red,challenges.killsOnLanersEarlyJungleAsJungler_Blue,challenges.killsOnLanersEarlyJungleAsJungler_Red,challenges.killsOnOtherLanesEarlyJungleAsLaner_Blue,challenges.killsOnOtherLanesEarlyJungleAsLaner_Red,challenges.laneMinionsFirst10Minutes_Blue,challenges.laneMinionsFirst10Minutes_Red,challenges.laningPhaseGoldExpAdvantage_Blue,challenges.laningPhaseGoldExpAdvantage_Red,challenges.takedownsFirstXMinutes_Blue,challenges.takedownsFirstXMinutes_Red,challenges.turretPlatesTaken_Blue,challenges.turretPlatesTaken_Red
matchId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NA1_4635230277,0,0,0.0,2.0,0,0,1.0,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,2,2,1,1,47.50,56.0,0.0,0.0,1,0,1.0,1.0,1.0,1.0,221,189,0.0,0.0,13,21,14,4
NA1_4649504526,0,0,0.0,2.0,1,1,0.0,0.0,1.0,0.0,0.000000,841.555898,0.0,0.0,2,2,0,2,51.60,60.0,0.0,0.0,0,0,1.0,2.0,0.0,0.0,227,202,0.0,0.0,11,20,2,8
NA1_4649537869,0,0,0.0,1.0,1,1,0.0,0.0,1.0,0.0,745.648699,0.000000,0.0,0.0,2,2,1,1,64.00,60.0,0.0,0.0,2,0,2.0,0.0,0.0,0.0,198,218,0.0,0.0,16,26,14,5
NA1_4650646767,0,0,2.0,0.0,0,1,0.0,0.0,0.0,1.0,854.821708,0.000000,0.0,0.0,2,2,2,0,56.00,32.0,1.0,0.0,0,0,1.0,2.0,0.0,0.0,220,225,1.0,0.0,24,35,9,6
NA1_4652371296,0,0,0.0,0.0,0,0,1.0,1.0,1.0,0.0,908.188676,0.000000,0.0,0.0,2,2,1,1,60.00,36.0,0.0,0.0,0,0,3.0,0.0,1.0,1.0,221,205,1.0,0.0,26,14,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA1_4701203108,0,0,0.0,0.0,0,1,0.0,1.0,0.0,0.0,0.000000,675.699342,1.0,0.0,2,2,1,1,44.80,61.2,0.0,1.0,2,8,1.0,2.0,0.0,2.0,197,193,0.0,3.0,13,47,9,28
NA1_4701204143,0,0,0.0,1.0,0,0,2.0,1.0,0.0,0.0,0.000000,684.083222,0.0,0.0,2,2,1,1,51.75,60.0,0.0,0.0,0,1,1.0,1.0,0.0,0.0,233,203,0.0,0.0,22,28,8,8
NA1_4701205902,0,0,1.0,0.0,1,1,0.0,0.0,0.0,1.0,767.608851,0.000000,0.0,0.0,3,1,2,0,55.60,44.4,0.0,0.0,1,0,2.0,2.0,3.0,0.0,216,185,1.0,0.0,35,25,11,4


In [25]:
#dropping mutually exclusive columns

df_total_pv = df_total_pv.drop(columns=['challenges.fasterSupportQuestCompletion_Red','challenges.initialBuffCount_Red',
                                       'challenges.initialCrabCount_Red','challenges.firstTurretKilledTime_Red'])

In [26]:
#for joining dataframes
df_team['matchId'] = match_id

In [27]:
#selecting 'first' columns and joining on match_id
df_team_first= df_team[df_team['teamId'] == 100].drop_duplicates()[['matchId','objectives.baron.first','objectives.champion.first',
                                                    'objectives.dragon.first','objectives.inhibitor.first',
                                                    'objectives.riftHerald.first','win']]
df_team_first = df_team_first.set_index('matchId')

creating the final dataframe which we will use to see if blue won or not(red won)

In [28]:
df_blue = df_total_pv.join(df_team_first,how='inner')
df_blue

,challenges.acesBefore15Minutes_Blue,challenges.acesBefore15Minutes_Red,challenges.baronBuffGoldAdvantageOverThreshold_Blue,challenges.baronBuffGoldAdvantageOverThreshold_Red,challenges.completeSupportQuestInTime_Blue,challenges.completeSupportQuestInTime_Red,challenges.earlyLaningPhaseGoldExpAdvantage_Blue,challenges.earlyLaningPhaseGoldExpAdvantage_Red,challenges.fasterSupportQuestCompletion_Blue,challenges.firstTurretKilledTime_Blue,challenges.getTakedownsInAllLanesEarlyJungleAsLaner_Blue,challenges.getTakedownsInAllLanesEarlyJungleAsLaner_Red,challenges.initialBuffCount_Blue,challenges.initialCrabCount_Blue,challenges.jungleCsBefore10Minutes_Blue,challenges.jungleCsBefore10Minutes_Red,challenges.junglerKillsEarlyJungle_Blue,challenges.junglerKillsEarlyJungle_Red,challenges.kTurretsDestroyedBeforePlatesFall_Blue,challenges.kTurretsDestroyedBeforePlatesFall_Red,challenges.killsOnLanersEarlyJungleAsJungler_Blue,challenges.killsOnLanersEarlyJungleAsJungler_Red,challenges.killsOnOtherLanesEarlyJungleAsLaner_Blue,challenges.killsOnOtherLanesEarlyJungleAsLaner_Red,challenges.laneMinionsFirst10Minutes_Blue,challenges.laneMinionsFirst10Minutes_Red,challenges.laningPhaseGoldExpAdvantage_Blue,challenges.laningPhaseGoldExpAdvantage_Red,challenges.takedownsFirstXMinutes_Blue,challenges.takedownsFirstXMinutes_Red,challenges.turretPlatesTaken_Blue,challenges.turretPlatesTaken_Red,objectives.baron.first,objectives.champion.first,objectives.dragon.first,objectives.inhibitor.first,objectives.riftHerald.first,win
matchId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NA1_4635230277,0,0,0.0,2.0,0,0,1.0,0.0,1.0,0.000000,0.0,0.0,2,1,47.50,56.0,0.0,0.0,1,0,1.0,1.0,1.0,1.0,221,189,0.0,0.0,13,21,14,4,0,0,0,0,1,0
NA1_4649504526,0,0,0.0,2.0,1,1,0.0,0.0,1.0,0.000000,0.0,0.0,2,0,51.60,60.0,0.0,0.0,0,0,1.0,2.0,0.0,0.0,227,202,0.0,0.0,11,20,2,8,0,0,1,0,0,0
NA1_4649537869,0,0,0.0,1.0,1,1,0.0,0.0,1.0,745.648699,0.0,0.0,2,1,64.00,60.0,0.0,0.0,2,0,2.0,0.0,0.0,0.0,198,218,0.0,0.0,16,26,14,5,0,1,0,0,1,0
NA1_4650646767,0,0,2.0,0.0,0,1,0.0,0.0,0.0,854.821708,0.0,0.0,2,2,56.00,32.0,1.0,0.0,0,0,1.0,2.0,0.0,0.0,220,225,1.0,0.0,24,35,9,6,1,0,1,1,1,1
NA1_4652371296,0,0,0.0,0.0,0,0,1.0,1.0,1.0,908.188676,0.0,0.0,2,1,60.00,36.0,0.0,0.0,0,0,3.0,0.0,1.0,1.0,221,205,1.0,0.0,26,14,6,8,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA1_4701203108,0,0,0.0,0.0,0,1,0.0,1.0,0.0,0.000000,1.0,0.0,2,1,44.80,61.2,0.0,1.0,2,8,1.0,2.0,0.0,2.0,197,193,0.0,3.0,13,47,9,28,0,0,0,0,0,0
NA1_4701204143,0,0,0.0,1.0,0,0,2.0,1.0,0.0,0.000000,0.0,0.0,2,1,51.75,60.0,0.0,0.0,0,1,1.0,1.0,0.0,0.0,233,203,0.0,0.0,22,28,8,8,0,1,0,0,0,0
NA1_4701205902,0,0,1.0,0.0,1,1,0.0,0.0,0.0,767.608851,0.0,0.0,3,2,55.60,44.4,0.0,0.0,1,0,2.0,2.0,3.0,0.0,216,185,1.0,0.0,35,25,11,4,1,0,0,1,1,1


## Train/Test Split


In [29]:
X = df_blue.drop(columns=['win'])
y= df_blue['win']
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=True,test_size=0.2,random_state=42)

In [30]:
X_train

,challenges.acesBefore15Minutes_Blue,challenges.acesBefore15Minutes_Red,challenges.baronBuffGoldAdvantageOverThreshold_Blue,challenges.baronBuffGoldAdvantageOverThreshold_Red,challenges.completeSupportQuestInTime_Blue,challenges.completeSupportQuestInTime_Red,challenges.earlyLaningPhaseGoldExpAdvantage_Blue,challenges.earlyLaningPhaseGoldExpAdvantage_Red,challenges.fasterSupportQuestCompletion_Blue,challenges.firstTurretKilledTime_Blue,challenges.getTakedownsInAllLanesEarlyJungleAsLaner_Blue,challenges.getTakedownsInAllLanesEarlyJungleAsLaner_Red,challenges.initialBuffCount_Blue,challenges.initialCrabCount_Blue,challenges.jungleCsBefore10Minutes_Blue,challenges.jungleCsBefore10Minutes_Red,challenges.junglerKillsEarlyJungle_Blue,challenges.junglerKillsEarlyJungle_Red,challenges.kTurretsDestroyedBeforePlatesFall_Blue,challenges.kTurretsDestroyedBeforePlatesFall_Red,challenges.killsOnLanersEarlyJungleAsJungler_Blue,challenges.killsOnLanersEarlyJungleAsJungler_Red,challenges.killsOnOtherLanesEarlyJungleAsLaner_Blue,challenges.killsOnOtherLanesEarlyJungleAsLaner_Red,challenges.laneMinionsFirst10Minutes_Blue,challenges.laneMinionsFirst10Minutes_Red,challenges.laningPhaseGoldExpAdvantage_Blue,challenges.laningPhaseGoldExpAdvantage_Red,challenges.takedownsFirstXMinutes_Blue,challenges.takedownsFirstXMinutes_Red,challenges.turretPlatesTaken_Blue,challenges.turretPlatesTaken_Red,objectives.baron.first,objectives.champion.first,objectives.dragon.first,objectives.inhibitor.first,objectives.riftHerald.first
matchId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NA1_4697583411,0,0,1.0,0.0,1,1,0.0,0.0,1.0,0.000000,0.0,0.0,3,1,60.00,56.0,0.0,0.0,0,1,1.0,1.0,0.0,0.0,210,233,0.0,0.0,33,20,3,9,1,1,1,0,1
NA1_4700767210,0,0,1.0,0.0,0,0,0.0,0.0,0.0,0.000000,0.0,0.0,1,1,48.00,52.0,0.0,0.0,0,0,1.0,0.0,0.0,1.0,224,232,0.0,0.0,24,21,9,6,1,0,0,1,1
NA1_4693209283,0,0,0.0,1.0,0,0,0.0,0.0,0.0,0.000000,0.0,0.0,2,1,52.00,63.0,0.0,0.0,1,2,2.0,1.0,0.0,0.0,208,220,0.0,1.0,27,36,9,7,0,1,1,0,0
NA1_4685662102,0,0,0.0,1.0,0,1,0.0,1.0,0.0,895.277679,0.0,1.0,2,0,63.75,64.0,0.0,0.0,0,0,0.0,3.0,1.0,0.0,211,223,0.0,0.0,9,24,3,8,0,0,0,0,0
NA1_4695835215,0,0,1.0,0.0,1,2,0.0,0.0,1.0,0.000000,0.0,0.0,1,1,63.60,40.4,0.0,0.0,0,0,2.0,0.0,0.0,0.0,224,251,1.0,0.0,21,17,3,4,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA1_4699119423,0,0,0.0,0.0,1,0,3.0,1.0,1.0,790.019643,0.0,0.0,1,1,46.00,59.0,0.0,0.0,2,0,0.0,2.0,2.0,0.0,190,244,0.0,1.0,34,16,21,6,0,0,0,0,1
NA1_4698391501,0,0,0.0,0.0,1,1,1.0,0.0,1.0,721.714685,0.0,0.0,2,1,50.00,54.0,1.0,1.0,2,0,4.0,2.0,0.0,0.0,201,216,1.0,0.0,39,21,10,11,0,1,1,0,1
NA1_4698635693,0,0,0.0,0.0,0,1,1.0,0.0,0.0,849.686174,0.0,1.0,2,1,52.00,64.0,0.0,0.0,0,0,1.0,1.0,1.0,2.0,223,180,1.0,0.0,34,22,12,2,0,0,1,1,1


## Parameter Tuning and Validation

In [31]:
#evaluating the model-building process
df_scores = pd.DataFrame(columns=['score','params'])
cv_df = pd.DataFrame()
for train_idx,validation_idx in KFold(n_splits=5,shuffle=True).split(X_train,y_train):
    
    
    X_ttrain,y_ttrain = X_train.iloc[train_idx],y_train.iloc[train_idx]

    X_validation,y_validation = X_train.iloc[validation_idx],y_train.iloc[validation_idx]

    
    paramgrid = {'max_depth':[None,1,3,5],
                'n_estimators':[100,200,300,400,500],
                'min_samples_split':[2,3,5,8,11],
                'min_samples_leaf':[1,3,5,7,9]}
    
    innercv = KFold(n_splits=5,shuffle=True)
    
    grid = GridSearchCV(estimator=RandomForestClassifier(),param_grid=paramgrid,cv=innercv,scoring='accuracy',refit=True,n_jobs=-1)
    
    result = grid.fit(X_ttrain,y_ttrain)
    
    best_p = result.best_params_

    #evaluating on the outerfolds
    model = RandomForestClassifier(oob_score=True,**best_p).fit(X_ttrain,y_ttrain)

    model.fit(X_ttrain,y_ttrain)

    score = model.oob_score_
    accuracy = accuracy_score(y_validation,model.predict(X_validation))
    
    #results of the best models (outer cv)

    df_scores = df_scores.append({'score':score,'score_acc':accuracy,'params':best_p},ignore_index=True)
    
    #results of all the models (inner cv)
    
    cv_result = pd.DataFrame.from_dict(result.cv_results_)[['params','mean_test_score','std_test_score']]
    
    
    cv_df = pd.concat([cv_df,cv_result],ignore_index=True)

In [32]:
df_scores

,score,params,score_acc
0,0.936964,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.918507
1,0.935631,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.933740
2,0.931632,"{'max_depth': None, 'min_samples_leaf': 1, 'mi...",0.943641
3,0.932584,"{'max_depth': None, 'min_samples_leaf': 3, 'mi...",0.931455
4,0.931074,"{'max_depth': None, 'min_samples_leaf': 3, 'mi...",0.938262


In [33]:
cv_df.groupby(cv_df['params'].astype('str')).mean().sort_values(by='mean_test_score',ascending=False)

,mean_test_score,std_test_score
params,,
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}",0.935748,0.006872
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 11, 'n_estimators': 100}",0.935709,0.006724
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 300}",0.935671,0.006633
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 400}",0.935633,0.007349
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 500}",0.935595,0.006595
...,...,...
"{'max_depth': 1, 'min_samples_leaf': 7, 'min_samples_split': 2, 'n_estimators': 100}",0.892521,0.020600
"{'max_depth': 1, 'min_samples_leaf': 5, 'min_samples_split': 3, 'n_estimators': 100}",0.892066,0.020038
"{'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 100}",0.891070,0.018718


In [34]:
cv_df['mean_test_score'].mean()

0.9206687423678129

the scores from different models evaluated and aggregated in the nested_gridsearchcv shows that the model building process is fairly consistent showing no volatility in in the accuracy score or in other words the model-building process managed to generalize well irregardless of the different folds it was validated/tested on

In [35]:
#running hyperparam tunning on the full training data

paramgrid = {'max_depth':[None,1,3,5],
                'n_estimators':[100,200,300,400,500],
                'min_samples_split':[2,3,5,8,11],
                'min_samples_leaf':[1,3,5,7,9]}


grid = GridSearchCV(estimator=RandomForestClassifier(),param_grid=paramgrid,
                    cv= KFold(n_splits=5,shuffle=True),scoring='accuracy',refit=True,n_jobs=-1)

result = grid.fit(X_train,y_train)

cv_result = pd.DataFrame.from_dict(result.cv_results_)[['params','mean_test_score','std_test_score']]

In [36]:
cv_result.groupby(cv_result['params'].astype('str')).mean().sort_values(by='mean_test_score',ascending=False)[:10]

,mean_test_score,std_test_score
params,,
"{'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 200}",0.937537,0.005811
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 200}",0.936775,0.007394
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}",0.936775,0.005874
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 11, 'n_estimators': 400}",0.936623,0.005918
"{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 400}",0.936471,0.004468
"{'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 500}",0.936471,0.005454
"{'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 11, 'n_estimators': 200}",0.936318,0.006783
"{'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 400}",0.936318,0.006230
"{'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 100}",0.936166,0.005363


In [37]:
#training the model with the best set of hyperparams
model = RandomForestClassifier(**result.best_params_)
model.fit(X_train,y_train)

#testing the model

predictions = model.predict(X_test)
proba_predictions = model.predict_proba(X_test)


score_accuracy = accuracy_score(y_test,predictions)
score_recall = recall_score(y_test,predictions)
score_auc = roc_auc_score(y_test,proba_predictions[:,1])
score_precision = precision_score(y_test,predictions)
print(f'Accuracy:{score_accuracy}')
print(f'Recall:{score_recall}')
print(f'Precision:{score_precision}')
print(f'ROC:{score_auc}')

Accuracy:0.9195612431444241
Recall:0.9098765432098765
Precision:0.9258793969849246
ROC:0.9702916313826864


## Limitations
1- Interpretability of results

the properties in the game that are reflected in the features of the data can each have a sort of a unique nature, while some features like the ones starting with "first" are of exclusive nature(either one team/player or the other got it) some other features do not fully reflect this property like "challenges.jungleCsBefore10Minutes" which leaves me with one of 2 choices, i either fully drop the features of one team and purely predict on the other and this case i would be losing some of the predictive power but gain interpretability, or i do the opposite(in either case mutually fully exclusive features can be dropped for one of the teams)

2-emulation of incomplete games

the only way it was possible to create a model that truly predicts an outcome of a game instead of guessing it after the fact was by only including the features known to have been defnitely logged before the conclusion of the game, as most of the features i found with "challenges" log data up to mostly either 7 or 10 or 15 minutes i couldnt have any games that lasted less than that and likewise if i wanted to predict a game that lasted less than that i would be unable with the data at hand  